# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [3]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

# Configure gmaps
gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [4]:
weather_data = "./WeatherPy.csv"
vacation_df = pd.read_csv(weather_data)
print(len(vacation_df))
vacation_df.head()

569


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Hirara,24.8000,125.2833,80.28,89,97,8.28,JP,1666383577
1,Saint Paul Harbor,57.7900,-152.4072,49.51,46,0,17.27,US,1666383578
2,Walvis Bay,-22.9575,14.5053,61.75,82,1,9.22,NaN,1666383578
3,Turukhansk,65.8167,87.9833,33.91,98,100,18.41,RU,1666383578
4,Nikolskoye,59.7035,30.7861,42.69,93,100,3.27,RU,1666383579


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [5]:
fig = gmaps.figure()

#Use the Lat and Lng as locations and Humidity as the weight.
#store lat / long data
locations = vacation_df[["Lat", "Lng"]].astype(float)
humidity = vacation_df["Humidity"].astype(float)

In [13]:
figure_layout = {
'width': '900px',
'height': '900px',
'border': '2px solid black',
'padding': '1px',
'margin': '0 auto 0 auto'
}

#create a heat map by location
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=120,
                                 point_radius = 1)

fig = gmaps.figure(layout=figure_layout)
fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(border='2px solid black', height='900px', margin='0 auto 0 auto', padding='1px', wi…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [20]:
#Drop rows that have null values
new_vacation_data = vacation_df.dropna()
#set the temerature range of the ideal location
clean_vacation_data = new_vacation_data[(new_vacation_data['Max Temp'] >= 65) 
                                        & (new_vacation_data['Max Temp'] < 80)
                                        & (new_vacation_data['Wind Speed'] < 10)
                                        & (new_vacation_data['Cloudiness'] == 0)]
print(len(clean_vacation_data))
clean_vacation_data.head()

39


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
79,Tura,25.5198,90.2201,67.64,77,0,5.84,IN,1666383602
98,Chhāpar,27.8167,74.4000,77.04,30,0,6.04,IN,1666383611
126,Rome,34.2570,-85.1647,73.90,28,0,3.44,US,1666383606
128,Mount Isa,-20.7333,139.5000,74.30,32,0,8.23,AU,1666383625
139,Gachsaran,30.3586,50.7981,71.69,53,0,0.00,IR,1666383629


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [49]:
hotel_df = pd.DataFrame()
hotel_df['City'] = ""
hotel_df['Country'] = ""
hotel_df['Lat'] = ""
hotel_df['Lng'] = ""
hotel_df['Hotel Name'] = ""

hotel_df

,City,Country,Lat,Lng,Hotel Name


In [69]:
test_data = clean_vacation_data.head()
test_data

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
79,Tura,25.5198,90.2201,67.64,77,0,5.84,IN,1666383602
98,Chhāpar,27.8167,74.4000,77.04,30,0,6.04,IN,1666383611
126,Rome,34.2570,-85.1647,73.90,28,0,3.44,US,1666383606
128,Mount Isa,-20.7333,139.5000,74.30,32,0,8.23,AU,1666383625
139,Gachsaran,30.3586,50.7981,71.69,53,0,0.00,IR,1666383629


In [86]:
# Loop through the clean_vacation_data and grab the lat/long informatin for each city

for index, row in test_data.iterrows():
    travel_location = str(test_data.loc[index,'City'] + "," + test_data.loc[index, 'Country'])
    
    base_url = f"https://maps.googleapis.com/maps/api/geocode/json?address={travel_location}&key={g_key}"
    # make request
    
    try:
        lat_long_data = requests.get(base_url).json()
    
    except Exception as e:
        
        print("did not find")
        
    city = lat_long_data["results"][0]["address_components"][0]["long_name"]
    country = lat_long_data["results"][0]["address_components"][2]["long_name"]
    lat = lat_long_data["results"][0]["geometry"]["location"]["lat"]
    lng = lat_long_data["results"][0]["geometry"]["location"]["lng"]
    
    
    hotel_df.loc[index,'City'] = city
    hotel_df.loc[index,'Country'] = country
    hotel_df.loc[index, "Lat"] = lat
    hotel_df.loc[index, "Lng"] = lng
    

In [87]:
hotel_df

,City,Country,Lat,Lng,Hotel Name
139,Gachsaran,Iran,30.365976,50.79363,NaN
79,Tura,Meghalaya,25.514071,90.203214,NaN
98,Chhapar,Rajasthan,27.818593,74.437907,NaN
126,Rome,Georgia,34.257038,-85.164673,NaN
128,Mount Isa,Queensland,-20.726369,139.493016,NaN


In [78]:
# Loop through the clean_vacation_data and grab the lat/long informatin for each city

for index, row in test_data.iterrows():

    travel_location = str(test_data.loc[index,'City'] + "," + test_data.loc[index, 'Country']) 
    print(travel_location)

Tura,IN
Chhāpar,IN
Rome,US
Mount Isa,AU
Gachsaran,IR


In [70]:
    base_url = f"https://maps.googleapis.com/maps/api/geocode/json?address={travel_location}&key={g_key}"
               
    # make request
    lat_long_data = requests.get(base_url).json()
    results = lat_long_data["results"] 
    try: 
        first_result = results[0]
    
    except (IndexError ):
            print(f' No {City} found.')
    try: #grab city name 
        hotel_df.loc[index,'City'] = first_result.get("long_name")

    except (KeyError, IndexError):
                                                              
        print("did not find")
                                                      
    try: #grab country 
        hotel_df.loc[index,'Country'] = first_result.get("short_name")
    
    except (KeyError, IndexError):
        
        print("did not find") 
                                                      
    try: #grab lat
                                                      
        hotel_df.loc[index, "Lat"] = first_result.get("lat") 
                                                      
    except (KeyError, IndexError):
        
        print("did not find")                                                      
                                                      
    try: #grab long
                                                      
        hotel_df.loc[index, "Lng"] = first_result.get("lng") 

    except (KeyError, IndexError):
        
        print("did not find")                                                  
    
    #check if there is a first item in the list 
    #check to see if length of result len(result) is greater than vero
    

In [73]:
print(json.dumps(Lat_Long_data, indent=4 , sort_keys=True))

{
    "results": [
        {
            "address_components": [
                {
                    "long_name": "Gachsaran",
                    "short_name": "Gachsaran",
                    "types": [
                        "locality",
                        "political"
                    ]
                },
                {
                    "long_name": "Kohgiluyeh and Boyer-Ahmad Province",
                    "short_name": "Kohgiluyeh and Boyer-Ahmad Province",
                    "types": [
                        "administrative_area_level_1",
                        "political"
                    ]
                },
                {
                    "long_name": "Iran",
                    "short_name": "IR",
                    "types": [
                        "country",
                        "political"
                    ]
                }
            ],
            "formatted_address": "Gachsaran, Kohgiluyeh and Boyer-Ahmad Province, Iran",
         

In [81]:
hotel_df.loc[index,'City'] = lat_long_data["results"][0]["address_components"][0]["long_name"]
hotel_df.loc[index,'Country'] = lat_long_data["results"][0]["address_components"][2]["short_name"]
hotel_df.loc[index, "Lat"] = lat_long_data["results"][0]["geometry"]["location"]["lat"]
hotel_df.loc[index, "Lng"] = lat_long_data["results"][0]["geometry"]["location"]["lng"]

In [56]:
radius = 5000
target_search = 'hotel'
target_lat_lng = f'{hotel_df.loc[index,Lat]},{hotel_df.loc[index,Lng]}'
target_lat_lng                                 

NameError: name 'Lat' is not defined

In [ ]:
# base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
    "query": query,
    "location": f"{lat},{lng}",
    "radius": radius,
    "rankby": "distance",
    "key": g_key,
}
# run a request using our params dictionary
response = requests.get(base_url, params=params)
response = response.json()

In [ ]:
print(json.dumps(response, indent=4, sort_keys=True))

In [ ]:
#******* REMVOVE BEFORE SUBMIT********
# Create bank symbol layer
# bank_layer = gmaps.symbol_layer(
#     locations, fill_color='rgba(0, 150, 0, 0.4)',
#     stroke_color='rgba(0, 0, 150, 0.4)', scale=2,
#     info_box_content=[f"Bank amount: {bank}" for bank in bank_rate]
# )


# fig = gmaps.figure()
# fig.add_layer(bank_layer)

# fig

In [ ]:
#******************REMOVE BEFORE SUBMIT **********************
# Create a combined map
# fig = gmaps.figure()

# fig.add_layer(heat_layer)
# fig.add_layer(bank_layer)

# fig

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
#******************REMOVE BEFORE SUBMIT **********************
# coordinates = [
#     (38.8976763, -77.0365298),
#     (47.6062095, -122.3320708),
#     (40.7127753, -74.0059728),
#     (30.267153, -97.7430608),
#     (44.977753, -93.2650108),     
# ]
# markers = gmaps.marker_layer(coordinates)
# fig.add_layer(markers)
# fig

In [ ]:
# Add marker layer ontop of heat map


# Display figure
